In [1]:
from tunable_agents import utility, main, agent
import utils
import gin
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tunable_agents.environments.gathering_env import gathering_env
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

%matplotlib inline
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
for file in os.listdir("results/"):
    if os.path.isfile("results/" + file):
        shape = np.load("results/" + file).shape
        print(f"{file:<60}     {shape}")

# Scalar Utilities

In [2]:
path = "results/"

results_files = {"repl_model-repl_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "64_64_model-replication_env" in file)],
                 "repl_model-cum_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "64_64_model-cum_rewards_env" in file)],
                 "repl_model-thresh_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "64_64_model-threshold_env" in file)],
                 "repl_model-cum_thresh_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "64_64_model-cum_threshold_env" in file)],
                 "128_128_64_model-repl_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "128_128_64_model-replication_env" in file)],
                 "128_128_64_model-cum_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "128_128_64_model-cum_rewards_env" in file)],
                 "128_128_64_model-thresh_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "128_128_64_model-threshold_env" in file)],
                 "128_128_64_model-cum_thresh_env": [file for file in os.listdir(path) if (os.path.isfile(path + file) and "128_128_64_model-cum_threshold_env" in file)]
                 }


keys = sorted(results_files.keys())
results = dict()
for key in keys:
    results[key] = [np.load("results/" + file) for file in results_files[key]]

if False:
    for key in keys:
        results[key] = [result[result != -1] for result in results[key]]


In [3]:
intra_wds = dict()
for key in keys:
    intra_wds[key] = [wasserstein_distance(results[key][i], results[key][j])
                      for i in range(len(results[key])) for j in range(i+1, len(results[key]))]

inter_wds = dict()
for i in range(len(keys)):
    for j in range(i+1, len(keys)):
        key_1, key_2 = keys[i], keys[j]
        if key_1.split("-")[1] == key_2.split("-")[1]:
            inter_wds[key_1 + "--" + key_2] = [wasserstein_distance(result_1, result_2)
                                            for result_1 in results[key_1] for result_2 in results[key_2]]

for key in keys:
    mean_WD = np.round(np.mean(intra_wds[key]), 4)
    print(f"{key:<40}     {mean_WD}")

128_128_64_model-cum_env                     0.987
128_128_64_model-cum_thresh_env              1.5801
128_128_64_model-repl_env                    0.7244
128_128_64_model-thresh_env                  2.0162
repl_model-cum_env                           1.327
repl_model-cum_thresh_env                    2.3361
repl_model-repl_env                          1.2287
repl_model-thresh_env                        4.303


In [4]:

for key in inter_wds.keys():
    mean_WD = np.round(np.mean(inter_wds[key]), 4)
    print(f"{key:<70}     {mean_WD}")

128_128_64_model-cum_env--repl_model-cum_env                               3.1498
128_128_64_model-cum_thresh_env--repl_model-cum_thresh_env                 1.8499
128_128_64_model-repl_env--repl_model-repl_env                             2.9361
128_128_64_model-thresh_env--repl_model-thresh_env                         5.0472


In [5]:
print("{:^30}            {:^12}      {:^10}".format("Model", "Mean", "Standard Error"))
for key in keys:
    res = [np.mean(result) for result in results[key]]
    mean = np.round(np.mean(res), 4)
    std_err = np.round(np.std(res)/np.sqrt(len(res)), 4)
    print(f"{key:<40}     {str(mean):<12}       {str(std_err):<30}")

            Model                             Mean          Standard Error
128_128_64_model-cum_env                     14.9522            0.2754                        
128_128_64_model-cum_thresh_env              8.5854             0.485                         
128_128_64_model-repl_env                    14.3529            0.1786                        
128_128_64_model-thresh_env                  8.2447             0.642                         
repl_model-cum_env                           11.8223            0.4006                        
repl_model-cum_thresh_env                    7.4677             0.8625                        
repl_model-repl_env                          11.436             0.3531                        
repl_model-thresh_env                        3.4226             1.3389                        


In [6]:
for key in keys:
    print(key, [np.mean(result) for result in results[key]])

128_128_64_model-cum_env [13.910725, 15.17225, 15.2183, 14.432275, 14.9077, 16.07195]
128_128_64_model-cum_thresh_env [8.889875, 7.134875, 9.937125, 9.4408, 7.524575]
128_128_64_model-repl_env [13.912125, 14.477425, 13.91235, 14.568975, 15.14865, 14.0978]
128_128_64_model-thresh_env [5.01925, 7.8694, 9.491775, 9.701975, 9.0186, 8.367375]
repl_model-cum_env [11.6996, 12.63365, 11.8517, 10.0577, 13.19475, 11.496625]
repl_model-cum_thresh_env [8.764525, 7.88765, 8.67945, 4.539075]
repl_model-repl_env [12.474725, 10.209075, 10.3794, 11.6756, 11.603375, 12.273975]
repl_model-thresh_env [-2.852275, 4.206975, 1.9436, 6.92235, 3.726675, 6.588025]


# Stale Analysis

In [ ]:
path = "results/"

results_files = {"david": ["david_200221.npy", "david_210221.npy", "david_210221_2.npy"],
                 "my_env": ["my_210220.npy", "my_210221.npy", "my_210221_2.npy"],
                 "new": ["new_gath_repl_1.npy", "new_gath_repl_2.npy", "new_gath_repl_3.npy"],
                 "tfa": ["gathering_replication_1.npy", "gathering_replication_2.npy", "gathering_replication_3.npy",
                         "gathering_replication_4.npy"]


keys = sorted(results_files.keys())
results = dict()
for key in keys:
    results[key] = [np.load("results/" + file, allow_pickle=True) for file in results_files[key]]

if True:
    for key in keys:
        results[key] = [result[result != -1] for result in results[key]]


In [ ]:
'''
plt.figure(figsize=(15, 12))
i = 1
for result in results["my_env"] + results["david"] + results["tfa"][:-1]:
    plt.subplot(3, 3, i)
    plt.title(f"Number of sampled episodes: {result.shape[0]}")
    plt.hist(result, bins=200, density=True)
    i += 1

plt.tight_layout()
plt.show()
'''
zz =0